In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
!pip install -U imbalanced-learn==0.4.0

In [ ]:
!pip install pandas-profiling[notebook,html]


In [ ]:


!pip install fastai==0.7.0

In [ ]:
PATH='/kaggle/input/telco-customer-churn/'

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


# First loading and cleaning the data

I used fast.ai packedge to transform categorical columns and to handle nulls
    1. it transform categorical columns to column with code like yes,no->1,2
    2. for categorical variable it put the null value to another class and add a colunm category_is_null
    3. for continous variable it replace the null with the median and add is_null in a nother column

In [ ]:
df_raw = pd.read_csv(f'{PATH}WA_Fn-UseC_-Telco-Customer-Churn.csv', low_memory=False)


In [ ]:
df_raw.info()

In [ ]:
df_raw.head()

In [ ]:
df_raw.drop("customerID",axis=1,inplace=True)
train_cats(df_raw)

In [ ]:
df, y, nas = proc_df(df_raw, 'Churn')

In [ ]:
df.info()

In [ ]:
df.head().T

In [ ]:
df_raw["Churn"].value_counts()

not balanced data set for the Churn

# Here i tried random forest with class_weigh="palanced" to penalize miss classifying the minority class

In [ ]:
X_train,_x,y_train,_y=train_test_split(df,y,test_size=0.3)
X_valid,X_test,y_valid,y_test=train_test_split(_x,_y,test_size=0.1)
X_train.shape,y_train.shape,X_valid.shape,y_valid.shape,X_test.shape,y_test.shape

In [ ]:
def acc(x,y): return np.sum(x==y)/len(y)

def print_score(m):
    res = [acc(m.predict(X_train), y_train), acc(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

here i used class_weight="balanced" to benalize random forest without removing data.

In [ ]:
m = RandomForestClassifier(class_weight="balanced",n_jobs=-1,min_samples_leaf=5,n_estimators=80,max_features=0.5)
%time m.fit(X_train, y_train)
print_score(m)

In [ ]:
print(classification_report(y_valid, m.predict(X_valid)))

# Here i used random downsampling then random forest

In [ ]:
churn_no_index=np.random.choice(df_raw[df_raw.Churn=="No"].index,df_raw[df_raw.Churn=="Yes"].shape[0])

In [ ]:
all_indexes=np.concatenate((df_raw[df_raw.Churn=="Yes"].index.values,churn_no_index),axis=0)

In [ ]:
df_raw_equal_classes=df_raw.iloc[all_indexes]

In [ ]:
df_raw_equal_classes.Churn.value_counts()

In [ ]:
df, y, nas = proc_df(df_raw_equal_classes, 'Churn')

In [ ]:
X_train,_x,y_train,_y=train_test_split(df,y,test_size=0.3)
X_valid,X_test,y_valid,y_test=train_test_split(_x,_y,test_size=0.1)

In [ ]:
m = RandomForestClassifier(n_jobs=-1,min_samples_leaf=5,n_estimators=80,max_features=0.5)
%time m.fit(X_train, y_train)
print_score(m)

In [ ]:
print(classification_report(y_valid, m.predict(X_valid)))

In [ ]:
fi = rf_feat_importance(m, df)

In [ ]:
fi.plot("cols",'imp','barh',legend=False)

the f-score now is petter

# Here i tried different methods to downsample and upsample the dataset

In [ ]:
df_raw.Churn.value_counts()

I made the train,test and validation data be the same distripution,,,,train_test_split do this by default

In [ ]:
df, y, nas = proc_df(df_raw, 'Churn')
X_train_b,_x,y_train_b,_y=train_test_split(df,y,test_size=0.3)
X_valid,X_test,y_valid,y_test=train_test_split(_x,_y,test_size=0.1)


In [ ]:
sum(y_train)/len(y_train),sum(y)/len(y),sum(y_test)/len(y_test)

then under and over sampling the training data

In [ ]:
# check version number
import imblearn
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.under_sampling import OneSidedSelection
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
undersample = OneSidedSelection(n_neighbors=1, n_seeds_S=200)

X_train, y_train = undersample.fit_resample(X_train_b, y_train_b)
oversample = BorderlineSMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [ ]:
sum(y_train)/len(y_train)

In [ ]:
m = RandomForestClassifier(n_jobs=-1,min_samples_leaf=5,n_estimators=100,max_features=0.5)
%time m.fit(X_train, y_train)
print_score(m)

In [ ]:
print(classification_report(y_valid, m.predict(X_valid)))

the percision is low,so it seams to be that droping from class 0 and increasing class 1 make the model falsy classifay class 0 as 1.

so, next i will add small portion of class 1 and penalize making it false

In [ ]:
oversample = BorderlineSMOTE(sampling_strategy=0.5)
X_train, y_train = oversample.fit_resample(X_train_b, y_train_b)

In [ ]:
sum(y_train_b)/len(y_train_b),sum(y_train)/len(y_train)

In [ ]:
m = RandomForestClassifier(n_jobs=-1,min_samples_leaf=5,n_estimators=100,max_features=0.5,class_weight='balanced')
%time m.fit(X_train, y_train)
print_score(m)

In [ ]:
print(classification_report(y_valid, m.predict(X_valid)))

when doing first under sampling that doesn't delete any random sample that deleted some exapmples from the majorty class
then doing another random undersampling,
the score is bad

In [ ]:
undersample = OneSidedSelection(n_neighbors=1, n_seeds_S=200,sampling_strategy="majority")
X_train, y_train = undersample.fit_resample(X_train_b, y_train_b)
undersample2 = RandomUnderSampler()
X_train, y_train = undersample2.fit_resample(X_train, y_train)

In [ ]:
sum(y_train_b)/len(y_train_b),sum(y_train)/len(y_train)

In [ ]:
m = RandomForestClassifier(n_jobs=-1,min_samples_leaf=5,n_estimators=100,max_features=0.5,class_weight='balanced')
%time m.fit(X_train, y_train)
print_score(m)

In [ ]:
print(classification_report(y_valid, m.predict(X_valid)))

did another under sampling technique that do no loss in model performance but also bad results

In [ ]:
from imblearn.under_sampling import CondensedNearestNeighbour
undersample = CondensedNearestNeighbour(n_neighbors=1)
undersample2 = RandomUnderSampler()
X_train, y_train = undersample2.fit_resample(X_train, y_train)

In [ ]:
sum(y_train_b)/len(y_train_b),sum(y_train)/len(y_train)

In [ ]:
m = RandomForestClassifier(n_jobs=-1,min_samples_leaf=5,n_estimators=100,max_features=0.5,class_weight='balanced')
%time m.fit(X_train, y_train)
print_score(m)

In [ ]:
print(classification_report(y_valid, m.predict(X_valid)))

# Here i tried logistic regression

will enhance the features to fit logistic regression 

In [ ]:
df, y, nas = proc_df(df_raw, 'Churn')
df.columns

In [ ]:
cat_columns=["gender","SeniorCitizen","Partner","Dependents",
             "PhoneService","MultipleLines","InternetService",
"OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport"
,"StreamingTV","StreamingMovies","Contract","PaperlessBilling","PaymentMethod"
]
cont_columns=["tenure",'MonthlyCharges', 'TotalCharges']

In [ ]:
from sklearn.preprocessing import normalize

In [ ]:
cat=pd.get_dummies(df[cat_columns].applymap(str),drop_first=True)


In [ ]:
cont=normalize(df[cont_columns])
cont=pd.DataFrame( cont,columns=cont_columns)

In [ ]:
 df=pd.concat([cat,cont],axis=1)

In [ ]:
df.head().T

In [ ]:
df.info()

In [ ]:
corr=df.corr()

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
profile=ProfileReport(df.astype(float))

In [ ]:
profile

In [ ]:
to_drop=["InternetService_3","DeviceProtection_2","OnlineBackup_2","OnlineSecurity_2","PhoneService_2"
        ,"MultipleLines_2","StreamingMovies_2","StreamingTV_2","TechSupport_2"]

In [ ]:
df.drop(to_drop,inplace=True,axis=1)

In [ ]:
X_train,_x,y_train,_y=train_test_split(df,y,test_size=0.3)
X_valid,X_test,y_valid,y_test=train_test_split(_x,_y,test_size=0.1)

In [ ]:
sum(y_train)/len(y_train)

lets predict with logistic regression that is work for imbalanced classification.

In [ ]:
clf = LogisticRegression(class_weight="balanced").fit(X_train, y_train)
print_score(clf)

In [ ]:
print(classification_report(clf.predict(X_valid),y_valid))

not so good,let's down sample and try again.

In [ ]:
undersample = RandomUnderSampler()
X_train, y_train = undersample.fit_resample(X_train, y_train)

In [ ]:
sum(y_train)/len(y_train)

In [ ]:
clf = LogisticRegression().fit(X_train, y_train)
print_score(clf)

In [ ]:
print(classification_report(clf.predict(X_valid),y_valid))

# So the best from what i did is random forest with down sampling

I will use feature importance to describe the data orderd from most important to less important

# Describing the variables

In [ ]:
profile=ProfileReport(df_raw)

In [ ]:
profile

> 1. contract 

In [ ]:
plt.figure(figsize=(14,8))
plt.subplot(1,2,1)
df_raw.Contract.value_counts().plot.pie(autopct='%1.0f%%')
plt.subplot(1,2,2)
df_raw.groupby("Churn").Contract.value_counts().plot.bar()

55% of the people have month to month contract and about half of them want to leave.
most of the peoble that want to leave are in  month to month contract


> 2. tenure

In [ ]:
df_raw.tenure.hist(density=True)
df_raw.tenure.hist(df_raw["Churn"], density=True)
df_raw.tenure.hist(df_raw["Churn"],cumulative = True, density=True)

for the people that want to leave 80% of them have tenure value less than 40

> 3. MonthlyCharges

In [ ]:
df_raw.MonthlyCharges.hist(density=True)
df_raw.MonthlyCharges.hist(df_raw["Churn"],density=True)
df_raw.MonthlyCharges.hist(df_raw["Churn"],density=True,cumulative = True)

for the people that don't want to leave about 25% of them have monthly charge less then 25,and the other 75% have monthly charge of mean about to 80 this porpotion is normly distributed.
for the poeple who want to leave, about 50% of them have monthly charge more than 75.

> 4. TotalCharge

In [ ]:
df["Churn"]=df_raw["Churn"]
df.TotalCharges.hist(density=True)
df.TotalCharges.hist(df["Churn"],density=True)
df.TotalCharges.hist(df["Churn"],density=True,cumulative = True)
df.drop("Churn",axis=1,inplace=True)

> 5.TechSupport

In [ ]:
plt.figure(figsize=(14,8))
plt.subplot(1,2,1)
df_raw.TechSupport.value_counts().plot.pie(autopct='%1.0f%%')
plt.subplot(1,2,2)
df_raw.groupby("Churn").TechSupport.value_counts().plot.bar()

about 50% of the people have no technical support.

most of the people that want to leave have no technical support.

> 6. OnlineSecurity

In [ ]:
plt.figure(figsize=(14,8))
plt.subplot(1,2,1)
df_raw.OnlineSecurity.value_counts().plot.pie(autopct='%1.0f%%')
plt.subplot(1,2,2)
df_raw.groupby("Churn").OnlineSecurity.value_counts().plot.bar()

about 50% of the people have no security.

most of the people that want to leave have no.

> 7.PaymentMethod

In [ ]:
plt.figure(figsize=(14,8))
plt.subplot(1,2,1)
df_raw.PaymentMethod.value_counts().plot.pie(autopct='%1.0f%%')
plt.subplot(1,2,2)
df_raw.groupby("Churn").PaymentMethod.value_counts().plot.bar()

most of the people that want to leave have payment method electronic check

> > 8.InternetService

In [ ]:
plt.figure(figsize=(14,8))
plt.subplot(1,2,1)
df_raw.InternetService.value_counts().plot.pie(autopct='%1.0f%%')
plt.subplot(1,2,2)
df_raw.groupby("Churn").InternetService.value_counts().plot.bar()

most of the people that want to leave have fiber optics

> > 9.StreamingTV

In [ ]:
plt.figure(figsize=(14,8))
plt.subplot(1,2,1)
df_raw.StreamingTV.value_counts().plot.pie(autopct='%1.0f%%')
plt.subplot(1,2,2)
df_raw.groupby("Churn").StreamingTV.value_counts().plot.bar()

In [ ]:
df_raw.StreamingTV.value_counts()

about 1/3 of the people that want to stay have no StreamingTV,but v much small
percintage of the people that want to leave have no StreamingTV.this means no StreamingTV have no effect on people to leave perhapse it may make them more likely to stay

> 10 OnlineBackup

In [ ]:
plt.figure(figsize=(14,8))
plt.subplot(1,2,1)
df_raw.OnlineBackup.value_counts().plot.pie(autopct='%1.0f%%')
plt.subplot(1,2,2)
df_raw.groupby("Churn").OnlineBackup.value_counts().plot.bar()

like the above have no internet sevice has no affect the people how want to leave,and that is clear from the internent service feature as small portion of the people that want to leave have no internet service

> 11. PaperlessBilling

In [ ]:
plt.figure(figsize=(14,8))
plt.subplot(1,2,1)
df_raw.PaperlessBilling.value_counts().plot.pie(autopct='%1.0f%%')
plt.subplot(1,2,2)
df_raw.groupby("Churn").PaperlessBilling.value_counts().plot.bar()

for people who want to leave : the porpotion of be no for PaperlessBilling over being yes is less than it in the people who want to stay,

> 12. DeviceProtection

In [ ]:
plt.figure(figsize=(14,8))
plt.subplot(1,2,1)
df_raw.DeviceProtection.value_counts().plot.pie(autopct='%1.0f%%')
plt.subplot(1,2,2)
df_raw.groupby("Churn").DeviceProtection.value_counts().plot.bar()

it is more likely that the people that have no device protection to be want to leave than the people who have

> 13. gender

In [ ]:
plt.figure(figsize=(14,8))
plt.subplot(1,2,1)
df_raw.gender.value_counts().plot.pie(autopct='%1.0f%%')
plt.subplot(1,2,2)
df_raw.groupby("Churn").gender.value_counts().plot.bar()

the gender doesn't tell any information

> 14. Partner

In [ ]:
plt.figure(figsize=(14,8))
plt.subplot(1,2,1)
df_raw.Partner.value_counts().plot.pie(autopct='%1.0f%%')
plt.subplot(1,2,2)
df_raw.groupby("Churn").Partner.value_counts().plot.bar()

>15. MultipleLines

In [ ]:
plt.figure(figsize=(14,8))
plt.subplot(1,2,1)
df_raw.MultipleLines.value_counts().plot.pie(autopct='%1.0f%%')
plt.subplot(1,2,2)
df_raw.groupby("Churn").MultipleLines.value_counts().plot.bar()

doesn't tell information

In [ ]:
plt.figure(figsize=(14,8))
plt.subplot(1,2,1)
df_raw.StreamingMovies.value_counts().plot.pie(autopct='%1.0f%%')
plt.subplot(1,2,2)
df_raw.groupby("Churn").StreamingMovies.value_counts().plot.bar()

doesn't increase the information, so it's sound that the following columns doesn't tell much information as i taked them in the order of feature importance that came from random forest.